## Reconocimiento de Imágenes en CIFAR10
---

### Parte C


Diseñe, entrene y evalue una red neuronal con salida softmax para el problema CIFAR a partir de la representacion original de las imagenes (pıxeles RGB). Experimente con distintas arquitecturas y metodos de entrenamiento, midiendo el error de clasificacion sobre el conjunto de validacion. En base a esta ultima medida de desempeño, decida que modelo, de entre todos los evaluados, evaluara finalmente en el conjunto de test. Reporte y discuta los resultados obtenidos.(*) Se espera que logre obtener un error de pruebas menor o igual a 0.5.

> (*)Para empezar, puede considerar una pequeña arquitectura de 1 capa escondida con 50 neuronas y con funciones de activacion ReLu. Como metodo de entrenamiento puede usar BP estocastico o en mini-batches, con weight decay y tasa de aprendizaje decreciente.

|Simbolos |  Definición | Salida
| :-: | :-: | :-: |
|(Xtr,Ytr)|Imagenes y etiquetas de **entrenamiento**| Dos matrices |
|(Xt,Yt)|Imagenes y etiquetas de **pruebas**|Dos matrices |
|(Xv,Yv)|Imagenes y etiquetas como **conjunto de validacion**, es decir para tomar decisiones de diseño acerca del modelo. |Dos matrices(1)|
|XtrN|Imagenes **Normalizadas** (centradas y escaladas)|Dos matrices |

> (1) Este ultimo conjunto debe ser extraıdo desde el conjunto de entrenamiento original y no debe superar las 5000 imagenes.

In [6]:
import cPickle as pickle
import numpy as np
import os
from scipy.misc import imread

In [43]:
from sklearn import preprocessing
import pandas as pd

In [44]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.optimizers import SGD
from keras.utils import np_utils

In [45]:
def load_CIFAR_one(filename):
    with open(filename, 'rb') as f:
        datadict = pickle.load(f)
        X = datadict['data']
        Y = datadict['labels']
        Y = np.array(Y)
        return X, Y

In [46]:
def load_CIFAR10(PATH):
    xs = []
    ys = []
    for b in range(1,6):
        f = os.path.join(PATH, 'data_batch_%d' % (b, ))
        X, Y = load_CIFAR_one(f)
        xs.append(X)
        ys.append(Y)
    Xtr = np.concatenate(xs)
    Ytr = np.concatenate(ys)
    del X, Y
    Xt, Yt = load_CIFAR_one(os.path.join(PATH, 'test_batch'))
    return Xtr, Ytr, Xt, Yt

In [47]:
def validacion_CIFAR(_Xtr, _Ytr, _size):
    filas = np.random.randint(0,_Xtr.shape[0],_size)
    Xv = Xtr[filas]
    Yv = Ytr[filas]
    return Xv, Yv

In [70]:
def normalizar(_Xtr, _Ytr, _Xt, _Yt,_centrando,_escalado):
    print "_centrando ="+str(_centrando)+" y _escalado="+str(_escalado)
    scaler = preprocessing.StandardScaler(with_mean=_centrando, with_std=_centrando).fit(_Xtr)
    XtrN = scaler.transform(_Xtr)
    XtN  = scaler.transform(_Xt)
    YtrN = np_utils.to_categorical(_Ytr, 10)
    YtN = np_utils.to_categorical(_Yt, 10)
    del _Xtr, _Xt
    return XtrN,XtN,YtrN,YtN

### Parametros

In [78]:
_validacion = 5000 #tamaño del conjunto de validación
_centrando = True
_escalado  = True
_url = '/Users/hfarias/Doctorado/1-2017/NN/Parte4/data/'
_numeroCapas           = 1
_numeroNeuronasOcultas = 50
_learningRate          = 0.01
#_learningRate          = learRate(20)
_epochs                = 300
#_activation            = ['sigmoid','relu']
_activation            = 'relu'
_decay                 =1e-6
Xtr, Ytr, Xt, Yt = load_CIFAR10(_url)
Xv, Yv = validacion_CIFAR(Xtr, Ytr,_validacion)

In [79]:
print "Dimenciones dataset de imagenes y etiquetas de entrenamiento son Xtr = "+str(Xtr.shape)+" y  Ytr ="+str(len(Ytr))
print "Dimenciones dataset de imagenes y etiquetas de prueba son Xt = "+str(Xt.shape)+" y  Ytr ="+str(len(Yt))
print "Dimenciones dataset de imagenes y etiquetas para validación es Xv = "+str(Xv.shape)+" y  Yv ="+str(len(Yv))

Dimenciones dataset de imagenes y etiquetas de entrenamiento son Xtr = (50000, 3072) y  Ytr =50000
Dimenciones dataset de imagenes y etiquetas de prueba son Xt = (10000, 3072) y  Ytr =10000
Dimenciones dataset de imagenes y etiquetas para validación es Xv = (5000, 3072) y  Yv =5000


### Modelo

Si bien hemos usado StandardScaler  de sklearn para la normalización de los datos, es importante considerar que Keras también dispone de una librería para este fin , que es ese caso se usa dentro de la construcción del modelo.

```python
from keras.layers.normalization import BatchNormalization
```

<h3 align = 'center'> Resultados </h3>

|Neuronas | Capas escondidas| Funcion de activacion | epochs| Accuracy
| :-: | :-: | :-: | :-: |
|64|2|(sigmoidal,relu)|500| 44.609| 
|64 |1|(sigmoidal) |500|16.16|  
|50 | 2 | (sigmoidal,relu,relu) | 250 |45.60 | 
|100 | 3 | (relu,relu,relu)| 250 |48.94 |
|100 | 4 | (relu,relu,relu)| 250 |47.89 |

>Como podemos ver los mejores resultados se dieron con 3 capas ocultas de 100 neuronas cada una, donde la función de activación RELU es la que presento mejores resultados.   


Resultados Keras:

9568/10000 [===========================>..] - ETA: 0s('Accuracy:', 44.609999999999999)

9984/10000 [============================>.] - ETA: 0s('Accuracy:', 16.16)


 9312/10000 [==========================>...] - ETA: 0s ('Accuracy:', 45.609999999999999)
 
 9248/10000 [==========================>...] - ETA: 0s ('Accuracy:', 48.949999999999996)

In [89]:
def modelo(_numeroCapas,_numeroNeuronasOcultas,_learningRate,_epochs,_activation,_decay,_url,_centrando,_escalado):
    
    Xtr, Ytr, Xt, Yt = load_CIFAR10(_url)
    
    XtrN,XtN,YtrN,YtN = normalizar(Xtr, Ytr, Xt, Yt,_centrando,_escalado)
    
    model = Sequential()
    #Capa de entrada
    model.add(Dense(_numeroNeuronasOcultas, input_dim=Xtr.shape[1], kernel_initializer='uniform')) 
    model.add(Activation(_activation))
    #Capa(s) escondia(s)
    model.add(Dense(_numeroNeuronasOcultas, kernel_initializer='uniform'))
    model.add(Activation(_activation))
    
    model.add(Dense(_numeroNeuronasOcultas, kernel_initializer='uniform'))
    model.add(Activation(_activation))
    
    model.add(Dense(_numeroNeuronasOcultas, kernel_initializer='uniform'))
    model.add(Activation(_activation))
    
    model.add(Dense(_numeroNeuronasOcultas, kernel_initializer='uniform'))
    model.add(Activation(_activation))
    #Capa de salida
    model.add(Dense(10, kernel_initializer='uniform'))
    model.add(Activation('softmax'))
    
    #Tasa de aprendizaje
    sgd = SGD(lr=_learningRate)
    model.compile(optimizer=sgd,loss='categorical_crossentropy',metrics=['accuracy'])
    
    #hist = model.fit(XtrN,YtN, epochs=_epochs,verbose=0, validation_data=(_X_test_scaled.as_matrix(), _y_test_scaled.as_matrix()))
    hist = model.fit(XtrN,YtrN, epochs=_epochs,verbose=0,batch_size=32)
    score = model.evaluate(XtN, YtN)
    print("Accuracy:", score[1]*100)
    return hist

In [81]:
historial = modelo(_numeroCapas,_numeroNeuronasOcultas,_learningRate,_epochs,_activation,_decay,_url,_centrando,_escalado)

_centrando =True y _escalado=True
 9568/10000 [===========================>..] - ETA: 0s('Accuracy:', 44.609999999999999)


In [82]:
_validacion = 5000 #tamaño del conjunto de validación
_centrando = True
_escalado  = True
_url = '/Users/hfarias/Doctorado/1-2017/NN/Parte4/data/'
_numeroCapas           = 1
_numeroNeuronasOcultas = 64
_learningRate          = 0.01
#_learningRate          = learRate(20)
_epochs                = 500
#_activation            = ['sigmoid','relu']
_activation            = 'relu'
_decay                 =1e-6

In [83]:
historial = modelo(_numeroCapas,_numeroNeuronasOcultas,_learningRate,_epochs,_activation,_decay,_url,_centrando,_escalado)

_centrando =True y _escalado=True
 9984/10000 [============================>.] - ETA: 0s('Accuracy:', 16.16)


In [85]:
_validacion = 5000 #tamaño del conjunto de validación
_centrando = True
_escalado  = True
_url = '/Users/hfarias/Doctorado/1-2017/NN/Parte4/data/'
_numeroCapas           = 2
_numeroNeuronasOcultas = 50
_learningRate          = 0.01
#_learningRate          = learRate(20)
_epochs                = 250
#_activation            = ['sigmoid','relu']
_activation            = 'relu'
_decay                 =1e-6

In [86]:
historial = modelo(_numeroCapas,_numeroNeuronasOcultas,_learningRate,_epochs,_activation,_decay,_url,_centrando,_escalado)

_centrando =True y _escalado=True
 9312/10000 [==========================>...] - ETA: 0s ('Accuracy:', 45.609999999999999)


In [87]:
_validacion = 5000 #tamaño del conjunto de validación
_centrando = True
_escalado  = True
_url = '/Users/hfarias/Doctorado/1-2017/NN/Parte4/data/'
_numeroCapas           = 3
_numeroNeuronasOcultas = 100
_learningRate          = 0.01
#_learningRate          = learRate(20)
_epochs                = 250
#_activation            = ['sigmoid','relu']
_activation            = 'relu'
_decay                 =1e-6

In [88]:
historial = modelo(_numeroCapas,_numeroNeuronasOcultas,_learningRate,_epochs,_activation,_decay,_url,_centrando,_escalado)

_centrando =True y _escalado=True
 9248/10000 [==========================>...] - ETA: 0s ('Accuracy:', 48.949999999999996)


In [90]:
_validacion = 5000 #tamaño del conjunto de validación
_centrando = True
_escalado  = True
_url = '/Users/hfarias/Doctorado/1-2017/NN/Parte4/data/'
_numeroCapas           = 4
_numeroNeuronasOcultas = 100
_learningRate          = 0.01
#_learningRate          = learRate(20)
_epochs                = 250
#_activation            = ['sigmoid','relu']
_activation            = 'relu'
_decay                 =1e-6

In [91]:
historial = modelo(_numeroCapas,_numeroNeuronasOcultas,_learningRate,_epochs,_activation,_decay,_url,_centrando,_escalado)

_centrando =True y _escalado=True
 9888/10000 [============================>.] - ETA: 0s ('Accuracy:', 47.890000000000001)
